In [3]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [4]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='アンケート')
# データフレームを表示して確認
df.head()

,アンケート
0,「使った瞬間、髪がしっとりサラサラになり驚きました。香りも強すぎずリラックスできる香りで気に...
1,「洗い上がりはスッキリするのに、髪が全然きしまず指通りが良くなりました！乾燥しがちな髪質です...
2,「香りが少し甘めで、好みが分かれそうです。個人的には好きですが、甘い香りが苦手な方には少し強...
3,「ボトルのデザインがシンプルでおしゃれなので、バスルームが映える感じがします。使用感も良くて...
4,「使い始めてから頭皮のかゆみが減りました。敏感肌なので合わないシャンプーが多いのですが、これ...


In [8]:
# アンケートのを指定された言語に翻訳
def translate(text, language):
    # 2. データをLLM用にテキスト形式に変換
    # プロンプトの作成
    prompt_text = f"以下を{language}に翻訳してください。\n{text}"

    # 3. OpenAI APIの呼び出し
    try:
        # OpenAI APIで応答を生成
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt_text},
            ],
            max_tokens=100,  # 応答の長さを調整
            temperature=0.3  # 創造性の調整
        )
        # キーワード抽出結果を取得
        keywords = response.choices[0].message.content.strip()
        return keywords
    
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return "エラー"

In [9]:
# テストコードで確認
text = translate("pandasは、Excelファイル（`.xls`、`.xlsx` 形式）の読み書きに加え、データの集計や変換、フィルタリングといった複雑な操作を手軽に行えるライブラリです。", "English")
print(text)

Pandas is a library that allows for easy reading and writing of Excel files (in `.xls` and `.xlsx` formats), as well as performing complex operations such as data aggregation, transformation, and filtering.


In [ ]:
# ワークフロー化
print("処理を開始します。")

# Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='アンケート')

# A列の各行のアンケートに対して指定した言語に翻訳し、B列に書き込む
df['翻訳'] = df['アンケート'].apply(translate, language="English")

# 翻訳をExcelファイルに保存
df.to_excel("アンケート翻訳.xlsx", index=False)

print("Excelファイルに翻訳を保存しました。")

処理を開始します。
Excelファイルに翻訳を保存しました。


In [12]:
# 翻訳結果を確認
pd.read_excel('アンケート翻訳.xlsx').head()

,アンケート,翻訳
0,「使った瞬間、髪がしっとりサラサラになり驚きました。香りも強すぎずリラックスできる香りで気に...,"""I was surprised at how my hair became smooth ..."
1,「洗い上がりはスッキリするのに、髪が全然きしまず指通りが良くなりました！乾燥しがちな髪質です...,"""After washing, my hair feels refreshed, and i..."
2,「香りが少し甘めで、好みが分かれそうです。個人的には好きですが、甘い香りが苦手な方には少し強...,"""The fragrance is slightly sweet, which may be..."
3,「ボトルのデザインがシンプルでおしゃれなので、バスルームが映える感じがします。使用感も良くて...,"""The design of the bottle is simple and stylis..."
4,「使い始めてから頭皮のかゆみが減りました。敏感肌なので合わないシャンプーが多いのですが、これ...,"""Since I started using it, the itching on my s..."
